# Kaggle Playground - Season 4 Episode 
## Binary Classification of Insurance Cross Selling

Competion link - https://www.kaggle.com/competitions/playground-series-s4e8

### Steps
- Import the necessary libraries, packages and modules
- Read the datsets as data framers

### Understand the problem

- class is the target variable
- It determines the class of a mushroom depending on the given variables

In [1]:
# Import the necessary libraries, packages and modules

import warnings
warnings.filterwarnings('ignore')

# import dtale    # Use of a web progrm to analysis the data deeply
import lightgbm as lgb
import logging
import matplotlib.pyplot as plt
import numpy as np
import optuna
import os
import pandas as pd
import pickle
import seaborn as sns
# import statsmodels.api as sm
import tensorflow as tf
import time
import xgboost as xgb
# import zipfile

from imblearn.over_sampling import RandomOverSampler
from optuna.samplers import TPESampler
#from pandas_profiling import ProfileReport
from scipy.stats import randint, uniform
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import (BaggingClassifier, RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier, HistGradientBoostingClassifier)
from sklearn.feature_selection import chi2, RFE, SelectKBest, SelectFromModel  
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# from skopt import BayesSearchCV
from xgboost import XGBClassifier

sns.set()
%matplotlib inline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
start_time = time.time()

In [2]:
# Test to see if TensorFlow can utilize the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
train_df = pd.read_csv('train.csv', index_col = 'id')
test_df = pd.read_csv('test.csv', index_col = 'id')

train_df.head()

logger.info(f"Train data load completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Train data load completed. Time elapsed: 7.79 seconds


In [4]:
test_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,
3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,17.12,b,NaN,w,u,w,t,g,NaN,d,a
3116946,6.90,o,t,o,f,NaN,c,y,1.27,10.75,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a


In [5]:
# # Checking for data imbalance in training dataframe

# temp = train_df['class'].value_counts()    # Counts the number of ones and zeroes in the column
# temp_df = pd.DataFrame({'class' : temp.index, 'values' : temp.values})    # Creating a data frame with value and their counts
# plt.figure(figsize = (4, 3))
# sns.barplot(x = 'class', y = 'values', data = temp_df)
# plt.show()

In [6]:
# plt.pie(temp_df['values'], labels = [0, 1])
# plt.show()

### Identify the target variable and features

- class is the target variable
- It determines the class of a mushroom depending on the given variables

### Remove duplicate rows

- Checked the sum of duplicated rows in train and test datasets
- No dupllicated rows in train dataframe

In [7]:
# train_df.duplicated().sum()

In [8]:
# test_df.duplicated().sum()

### Handling missing values

- Checked the missing values in column
- There are considerable amount of missing values in many columns

In [9]:
train_df.shape

(3116945, 21)

In [10]:
test_df.shape

(2077964, 20)

In [11]:
# train_df.info()

In [12]:
# test_df.info()

In [13]:
# Since we have only one data set, spliting it into train and test (validation)

raw_train_df, validation_df = train_test_split(train_df, train_size = 0.75, random_state = 1, stratify = train_df['class'])
logger.info(f"Train test split completed. Time elapsed: {time.time() - start_time:.2f} seconds")
raw_train_df.head(2)

INFO:__main__:Train test split completed. Time elapsed: 16.24 seconds


,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,,
748879,p,2.45,x,g,n,f,a,NaN,p,7.12,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
589395,p,2.71,b,NaN,k,f,a,NaN,k,7.78,...,NaN,NaN,g,NaN,w,f,f,k,g,u


In [14]:
# raw_train_df.isnull().sum()

In [15]:
# test_df.isnull().sum()

In [16]:
# # Check for missing values

# missing_values = raw_train_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage = (missing_values / len(raw_train_df)) * 100
# missing_percentage

In [17]:
# # Check for missing values

# missing_values = validation_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage = (missing_values / len(validation_df)) * 100
# missing_percentage

In [18]:
# # Check for missing values

# missing_values_test = test_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage_test = (missing_values_test / len(test_df)) * 100
# missing_percentage_test

### Data Preprocessing
- Droping columns with more than 50% missing values
- Using simple imputer
- Encoding Categorical Variables

In [19]:
# Drop columns with extremely high missing values
columns_to_drop = ['veil-type', 'spore-print-color', 'stem-root', 'veil-color', 'stem-surface']
raw_train_df.drop(columns=columns_to_drop, inplace=True)
validation_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

# # Check for missing values

# missing_values = train_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage = (missing_values / len(train_df)) * 100
# missing_percentage

In [20]:
# # Check for missing values

# missing_values_test = test_df.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage_test = (missing_values_test / len(test_df)) * 100
# missing_percentage_test

In [21]:
# Spliting dependent and independent valriable

y_train = raw_train_df['class']
raw_train_df = raw_train_df.drop('class', axis = 1)

raw_train_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
id,,,,,,,,,,,,,,,
748879,2.45,x,g,n,f,a,NaN,p,7.12,2.30,w,f,f,d,u
589395,2.71,b,NaN,k,f,a,NaN,k,7.78,2.33,g,f,f,g,u


In [22]:
# # Spliting dependent and independent valriable

y_val = validation_df['class']
validation_df = validation_df.drop('class', axis = 1)

validation_df.head(2)

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
id,,,,,,,,,,,,,,,
1706883,1.40,x,g,y,f,d,d,n,1.56,2.43,n,f,f,d,a
1554994,5.35,f,h,g,f,s,d,w,4.75,9.63,g,f,f,d,u


In [23]:
# y_train = raw_train_df['class']
# raw_train_df.drop('class', axis = 1, inplace = True)

# raw_train_df.head(2)

In [24]:
# y_val = validation_df['class']
# validation_df.drop('class', axis = 1, inplace = True)

# validation_df.head(2)

In [25]:
# Identify numerical and categorical columns
numerical_cols = raw_train_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = raw_train_df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print("Unique Value is each categorical column :")
    print(col, train_df[col].nunique())

Unique Value is each categorical column :
cap-shape 74
Unique Value is each categorical column :
cap-surface 83
Unique Value is each categorical column :
cap-color 78
Unique Value is each categorical column :
does-bruise-or-bleed 26
Unique Value is each categorical column :
gill-attachment 78
Unique Value is each categorical column :
gill-spacing 48
Unique Value is each categorical column :
gill-color 63
Unique Value is each categorical column :
stem-color 59
Unique Value is each categorical column :
has-ring 23
Unique Value is each categorical column :
ring-type 40
Unique Value is each categorical column :
habitat 52
Unique Value is each categorical column :
season 4


In [26]:
# Remove the first element
# categorical_cols = categorical_cols[1:]
# categorical_cols

In [27]:
# Define the imputer for numerical columns (median imputation)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Define a function to apply LabelEncoder to each categorical column
def encode_labels(df, columns):
    df_encoded = df.copy()
    le_dict = {}
    for col in columns:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])
        le_dict[col] = le
    return df_encoded, le_dict

# Define the imputer and encoder for categorical columns using LabelEncoder
def encode_labels_pipeline(X):
    df = pd.DataFrame(X, columns=categorical_cols)
    df_encoded, le_dict = encode_labels(df, categorical_cols)
    return df_encoded.values

# Create a ColumnTransformer for numerical and categorical transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', FunctionTransformer(func=lambda X: encode_labels_pipeline(X), validate=False), categorical_cols)
    ],
    remainder='passthrough'
)

# Fit the preprocessor on the training data
preprocessor.fit(raw_train_df)

# Transform both the training and test data
train_df_transformed = preprocessor.transform(raw_train_df)
validation_df_transformed = preprocessor.transform(validation_df)
test_df_transformed = preprocessor.transform(test_df)

print("Transformed Training Data Shape:", train_df_transformed.shape)
print("Transformed Validation Data Shape:", validation_df_transformed.shape)
print("Transformed Test Data Shape:", test_df_transformed.shape)

logger.info(f"Missing values and categorical columns treatment completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Missing values and categorical columns treatment completed. Time elapsed: 39.16 seconds


Transformed Training Data Shape: (2337708, 15)
Transformed Validation Data Shape: (779237, 15)
Transformed Test Data Shape: (2077964, 15)


In [28]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.fit_transform(y_val)

y_train

array([1, 1, 1, ..., 1, 0, 0])

In [29]:
def optimize_memory_usage(df):
    start_mem_usage = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type.name in ['category', 'object']:
            raise ValueError(f"Column '{col}' is of type '{col_type.name}'")

        c_min = df[col].min()
        c_max = df[col].max()
        
        if str(col_type)[:3] == 'int':
            
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
                
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
                
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
                
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)
        
        else:
        
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            
            else:
                df[col] = df[col].astype(np.float64)

    end_mem_usage = df.memory_usage().sum() / 1024**2
    print(f'------ Memory usage before: {start_mem_usage:.2f} MB')
    print(f'------ Memory usage after: {end_mem_usage:.2f} MB')
    print(f'------ Reduced memory usage by {(100 * (start_mem_usage - end_mem_usage) / start_mem_usage):.1f}%')
    print('**********************' * 5)

    return df

In [30]:
# Combine feature names
all_feature_names = numerical_cols.tolist() + categorical_cols.tolist()

# Convert to DataFrame
train_df_transformed = pd.DataFrame(train_df_transformed, columns=all_feature_names)
validation_df_transformed = pd.DataFrame(validation_df_transformed, columns=all_feature_names)
test_df_transformed = pd.DataFrame(test_df_transformed, columns=all_feature_names)

# # Check for missing values
# missing_values_train = train_df_transformed.isnull().sum()

# # Calculate the percentage of missing values
# missing_values_train = (missing_values_train / len(train_df_transformed)) * 100
# missing_values_train

In [31]:
train_df_transformed = optimize_memory_usage(train_df_transformed)
validation_df_transformed = optimize_memory_usage(validation_df_transformed)
test_df_transformed = optimize_memory_usage(test_df_transformed)

------ Memory usage before: 267.53 MB
------ Memory usage after: 66.88 MB
------ Reduced memory usage by 75.0%
**************************************************************************************************************
------ Memory usage before: 89.18 MB
------ Memory usage after: 22.29 MB
------ Reduced memory usage by 75.0%
**************************************************************************************************************
------ Memory usage before: 237.80 MB
------ Memory usage after: 59.45 MB
------ Reduced memory usage by 75.0%
**************************************************************************************************************


In [32]:
# # Check for missing values

# missing_values_test = test_df_transformed.isnull().sum()

# # Calculate the percentage of missing values

# missing_percentage_test = (missing_values_test / len(test_df_transformed)) * 100
# missing_percentage_test

In [33]:
train_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,-0.829102,0.285889,-1.093750,58.0,46.0,45.0,8.0,37.0,44.0,39.0,43.0,3.0,12.0,21.0,2.0
1,-0.773438,0.530273,-1.089844,36.0,70.0,42.0,8.0,37.0,44.0,34.0,27.0,3.0,12.0,25.0,2.0


In [34]:
validation_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,-1.054688,-1.773438,-1.077148,35.0,16.0,41.0,3.0,10.0,7.0,23.0,24.0,5.0,12.0,7.0,0.0
1,-0.206177,-0.591797,-0.188354,21.0,17.0,25.0,3.0,24.0,7.0,32.0,17.0,5.0,12.0,7.0,2.0


In [35]:
test_df_transformed.head(2)

,cap-diameter,stem-height,stem-width,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-color,has-ring,ring-type,habitat,season
0,0.500488,1.771484,0.736816,59.0,59.0,44.0,18.0,66.0,35.0,52.0,51.0,17.0,15.0,16.0,0.0
1,0.126831,-1.880859,-0.050049,50.0,53.0,45.0,5.0,66.0,17.0,54.0,38.0,6.0,14.0,16.0,0.0


In [54]:
# Feature Selection
def feature_selection(X_train, y_train, model):
    if hasattr(model, 'coef_') or hasattr(model, 'feature_importances_'):
        selector = SelectFromModel(model, threshold='mean')
        selector.fit(X_train, y_train)
        return selector.transform(X_train), selector.get_support()
    else:
        raise ValueError("Feature selection not supported for this model.")

def alternative_feature_selection(X_train, y_train):
    selector = SelectKBest(score_func=f_classif, k='all')
    X_train_selected = selector.fit_transform(X_train, y_train)
    return X_train_selected, selector.get_support()

# Initialize models
models = {
#    'Logistic Regression': LogisticRegression(),
#    'Ridge Classifier': RidgeClassifier(),
#    'Decision Tree': DecisionTreeClassifier(),
#    'Bagging Classifier': BaggingClassifier(),
#    'Random Forest': RandomForestClassifier(),
#    'AdaBoost Classifier': AdaBoostClassifier(),
#    'Gradient Boosting': GradientBoostingClassifier(),
#    'SVC': SVC(),
#    'KNN': KNeighborsClassifier(),
    'XGBoost': xgb.XGBClassifier(tree_method='gpu_hist'),
    'LightGBM': lgb.LGBMClassifier(device='gpu')
}

# Define parameter grids for RandomizedSearchCV
param_grids = {
    'Logistic Regression': {
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
        'penalty': ['l2', 'none'],  # 'elasticnet' is not supported by 'lbfgs'
        'C': uniform(0.001, 1000),  # Inverse of regularization strength
        'max_iter': [100, 200, 300]
    },
    'Ridge Classifier': {
        'alpha': uniform(0.1, 10),  # Regularization strength
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg'],
        'fit_intercept': [True, False],
        'max_iter': [100, 200, 300],
       # 'normalize': [True, False]  # This should be removed as it's not a valid parameter
    },
    'Decision Tree': {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': randint(3, 20),
    'min_samples_split': uniform(0.01, 0.1),  # Random float between 0.01 and 0.1
    'min_samples_leaf': uniform(0.01, 0.1),  # Random float between 0.01 and 0.1
    'min_weight_fraction_leaf': uniform(0.0, 0.1),  # Random float between 0.0 and 0.1
    'max_features': ['auto', 'sqrt', 'log2', None],
    'max_leaf_nodes': randint(10, 50),
    'min_impurity_decrease': uniform(0.0, 0.1),
    'class_weight': [None, 'balanced']
},
    'Bagging Classifier': {
    'base_estimator': [DecisionTreeClassifier(), None],  # Default is DecisionTreeClassifier
    'n_estimators': randint(10, 100),
    'max_samples': uniform(0.5, 0.5),  # Random float between 0.5 and 1.0
    'max_features': uniform(0.5, 0.5),  # Random float between 0.5 and 1.0
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
    'oob_score': [True, False],
    'n_jobs': [None, -1],
    'random_state': [42]  # Set to None for random results, or a fixed integer for reproducibility
},
    'Random Forest': {
    'n_estimators': randint(50, 300),
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, randint(3, 20)],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2', uniform(0.5, 0.5)],
    'bootstrap': [True, False],
    'oob_score': [True, False],
    'n_jobs': [None, -1],
    'random_state': [42],
    'verbose': [0, 1],
    'warm_start': [True, False],
    'class_weight': [None, 'balanced']
},
    'AdaBoost Classifier': {
    'base_estimator': [None, DecisionTreeClassifier(max_depth=1)],
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 1.0),
    'algorithm': ['SAMME', 'SAMME.R'],
    'random_state': [42]
},
    'Gradient Boosting': {
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 0.5),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['sqrt', 'log2', None, uniform(0.1, 0.9)],
    'subsample': uniform(0.5, 0.5),
    'criterion': ['friedman_mse', 'squared_error', 'mae'],
    #'alpha': uniform(0.5, 0.5),
    'random_state': [42],
    'verbose': [0, 1]
},
    'SVC': {
    'C': uniform(0.1, 10),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': randint(2, 5),
    'gamma': ['scale', 'auto', uniform(0.001, 1)],
    'coef0': uniform(0, 10),
    'shrinking': [True, False],
    'probability': [True, False],
    'tol': uniform(1e-5, 1e-1),
    'cache_size': uniform(50, 500),
    'class_weight': [None, 'balanced'],
    'verbose': [True, False],
    'max_iter': [-1, 100, 200],
    'decision_function_shape': ['ovr', 'ovo'],
    'break_ties': [True, False],
    'random_state': [42]
},
    'KNN': {
    'n_neighbors': randint(1, 30),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': randint(1, 3),
    'metric': ['minkowski', 'euclidean', 'manhattan', 'chebyshev', 'hamming'],
    'leaf_size': randint(10, 50),
    'n_jobs': [None, -1]
},
    'XGBoost': {
    'n_estimators': randint(100, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 0.5),
    'min_child_weight': randint(1, 10),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
    'scale_pos_weight': uniform(1, 10),
    'max_delta_step': randint(0, 10),
    'colsample_bylevel': uniform(0.5, 0.5),
    'colsample_bynode': uniform(0.5, 0.5)
},
    'LightGBM': {
    'num_leaves': randint(20, 100),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(100, 300),
    'min_child_samples': randint(10, 100),
    'min_split_gain': uniform(0, 0.5),
    'subsample': uniform(0.5, 0.5),
    'subsample_freq': randint(1, 10),
    'colsample_bytree': uniform(0.5, 0.5),
    'colsample_bylevel': uniform(0.5, 0.5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
    'scale_pos_weight': uniform(1, 10)
}
}

# Optuna objective function for XGBoost and LightGBM
def objective(trial, model_name, model, X_train, y_train, X_val, y_val):
    if model_name == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
            'subsample': trial.suggest_uniform('subsample', 0.7, 0.9),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 0.9),
            'gamma': trial.suggest_uniform('gamma', 0, 0.2),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
        }
    elif model_name == 'LightGBM':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
            'num_leaves': trial.suggest_int('num_leaves', 31, 127),
            'max_depth': trial.suggest_int('max_depth', -1, 10),
            'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 0.2),
            'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 0.2)
        }

    model.set_params(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    score = matthews_corrcoef(y_val, preds)
    return score

# Hyperparameter tuning using RandomizedSearchCV
def tune_model(model_name, model, param_grid, X_train, y_train):
    search = RandomizedSearchCV(
        model, param_distributions=param_grid, n_iter=10, scoring='f1', cv=3, random_state=42
    )
    search.fit(X_train, y_train)
    print(f"{model_name} Best Parameters: {search.best_params_}")  # Print best parameters
    return search.best_estimator_, search.best_params_

# Hyperparameter tuning using Optuna for XGBoost and LightGBM
def optuna_tune_model(model_name, model, X_train, y_train, X_val, y_val):
    study = optuna.create_study(direction='maximize', sampler=TPESampler())
    #study.optimize(lambda trial: objective(trial, model_name, model, None, X_train, y_train), n_trials=50)
    study.optimize(lambda trial: objective(trial, model_name, model, X_train, y_train, X_val, y_val), n_trials=50)
    print(f"{model_name} Best Parameters: {study.best_params}")  # Print best parameters
    return study.best_params

# Train and evaluate models
def evaluate_models(X_train, y_train, X_val, y_val):
    best_models = {}
    for name, model in models.items():
        logger.info(f"Evaluating {name}")
    
        # Feature Selection
        if hasattr(model, 'coef_') or hasattr(model, 'feature_importances_'):
            logger.info(f"Performing feature selection for {name}")
            X_train_selected, support = feature_selection(X_train, y_train, model)
            X_val_selected = X_val.iloc[:, support]
        else:
            logger.info(f"Skipping feature selection for {name} due to lack of support")
            X_train_selected = X_train
            X_val_selected = X_val

        if name in ['XGBoost', 'LightGBM']:
            # Hyperparameter tuning using Optuna
            #params = optuna_tune_model(name, model, X_train_selected, y_train)
            params = optuna_tune_model(name, model, X_train_selected, y_train, X_val_selected, y_val)
            model.set_params(**params)
            model.fit(X_train_selected, y_train)

        else:
            # Hyperparameter tuning using RandomizedSearchCV
            best_model, best_params = tune_model(name, model, param_grids[name], X_train_selected, y_train)
            model = best_model

        # Training and validation MCC score calculation
        train_preds = model.predict(X_train_selected)
        val_preds = model.predict(X_val_selected)

        train_score = matthews_corrcoef(y_train, train_preds)
        val_score = matthews_corrcoef(y_val, val_preds)

        logger.info(f"{name} - Train MCC Score: {train_score:.4f}")
        logger.info(f"{name} - Validation MCC Score: {val_score:.4f}")
        logger.info(f"Time elapsed: {time.time() - start_time:.2f} seconds")

        best_models[name] = model

    return best_models

# Testing best models on the test set
def test_best_models(best_models, X_test, y_test):
    results = {}
    for name, model in best_models.items():
        preds = model.predict(X_test)
        score = matthews_corrcoef(y_test, preds)
        results[name] = score
        logger.info(f"Test MCC Score for {name}: {score:.4f}")
    return results

INFO:__main__:Evaluating Logistic Regression
INFO:__main__:Skipping feature selection for Logistic Regression due to lack of support
Logistic Regression Best Parameters: {'C': 155.99552033620265, 'max_iter': 300, 'penalty': 'l2', 'solver': 'saga'}
INFO:__main__:Logistic Regression - Train MCC Score: 0.2348
INFO:__main__:Logistic Regression - Validation MCC Score: 0.0483
INFO:__main__:Time elapsed: 1058.59 seconds
INFO:__main__:Evaluating Ridge Classifier
INFO:__main__:Skipping feature selection for Ridge Classifier due to lack of support
Ridge Classifier Best Parameters: {'alpha': 1.9182496720710063, 'fit_intercept': True, 'max_iter': 100, 'solver': 'lsqr'}
INFO:__main__:Ridge Classifier - Train MCC Score: 0.2370
INFO:__main__:Ridge Classifier - Validation MCC Score: 0.0494
INFO:__main__:Time elapsed: 1115.98 seconds
INFO:__main__:Evaluating Decision Tree
INFO:__main__:Skipping feature selection for Decision Tree due to lack of support
Decision Tree Best Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 17, 'max_features': 'log2', 'max_leaf_nodes': 17, 'min_impurity_decrease': 0.05986584841970366, 'min_samples_leaf': 0.025601864044243652, 'min_samples_split': 0.025599452033620268, 'min_weight_fraction_leaf': 0.005808361216819946, 'splitter': 'random'}
INFO:__main__:Decision Tree - Train MCC Score: 0.0000
INFO:__main__:Decision Tree - Validation MCC Score: 0.0000
INFO:__main__:Time elapsed: 1155.78 seconds
INFO:__main__:Evaluating Bagging Classifier
INFO:__main__:Skipping feature selection for Bagging Classifier due to lack of support
Bagging Classifier Best Parameters: {'base_estimator': None, 'bootstrap': False, 'bootstrap_features': False, 'max_features': 0.6039708314340944, 'max_samples': 0.7838501639099957, 'n_estimators': 72, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
INFO:__main__:Bagging Classifier - Train MCC Score: 0.9990
INFO:__main__:Bagging Classifier - Validation MCC Score: 0.2880
INFO:__main__:Time elapsed: 8444.69 seconds
INFO:__main__:Evaluating Random Forest
INFO:__main__:Skipping feature selection for Random Forest due to lack of support
Random Forest Best Parameters: {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 171, 'n_jobs': None, 'oob_score': True, 'random_state': 42, 'verbose': 0, 'warm_start': True}
INFO:__main__:Random Forest - Train MCC Score: 0.9837
INFO:__main__:Random Forest - Validation MCC Score: 0.0909
INFO:__main__:Time elapsed: 17699.45 seconds
INFO:__main__:Evaluating AdaBoost Classifier
INFO:__main__:Skipping feature selection for AdaBoost Classifier due to lack of support
AdaBoost Classifier Best Parameters: {'algorithm': 'SAMME.R', 'base_estimator': None, 'learning_rate': 0.9799098521619943, 'n_estimators': 285, 'random_state': 42}
INFO:__main__:AdaBoost Classifier - Train MCC Score: 0.6587
INFO:__main__:AdaBoost Classifier - Validation MCC Score: 0.2890
INFO:__main__:Time elapsed: 35149.93 seconds
INFO:__main__:Evaluating Gradient Boosting
INFO:__main__:Skipping feature selection for Gradient Boosting due to lack of support
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.2727           0.1060            3.02m
         2           1.1973           0.0717            3.43m
         3           1.1580           0.0401            3.34m
         4           1.1148           0.0440            3.48m
         5           1.0666           0.0493            3.42m
         6           1.0115           0.0547            3.34m
         7           0.9581           0.0561            3.40m
         8           0.9246           0.0260            3.34m
         9           0.8920           0.0373            3.37m
        10           0.8556           0.0374            3.34m
        20           0.5829           0.0191            3.11m
        30           0.4556           0.0154            2.82m
        40           0.3691           0.0143            2.58m
        50           0.3024           0.0113            2.30m
        60           0.2535           0.0041            2.04m
        70           0.2170           0.0035            1.81m
        80           0.1905          -0.0009            1.56m
        90           0.1708          -0.0021            1.32m
       100           0.1604           0.0020            1.07m
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.2727           0.1050            3.75m
         2           1.1976           0.0746            3.76m
         3           1.1560           0.0404            3.55m
         4           1.0883           0.0677            3.63m
         5           1.0577           0.0303            3.40m
         6           1.0069           0.0516            3.23m
         7           0.9615           0.0458            3.28m
         8           0.9242           0.0326            3.26m
         9           0.8912           0.0385            3.30m
        10           0.8415           0.0457            3.31m
        20           0.5992           0.0184            3.11m
        30           0.4283           0.0179            2.85m
        40           0.3523           0.0007            2.62m
        50           0.2958           0.0056            2.34m
        60           0.2450           0.0000            2.07m
        70           0.2153           0.0009            1.82m
        80           0.1873          -0.0013            1.58m
        90           0.1679           0.0036            1.33m
       100           0.1550          -0.0013            1.08m
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.2733           0.1054            3.05m
         2           1.1978           0.0733            3.40m
         3           1.1583           0.0380            3.33m
         4           1.1150           0.0428            3.46m
         5           1.0668           0.0525            3.42m
         6           1.0116           0.0504            3.33m
         7           0.9576           0.0559            3.40m
         8           0.9240           0.0338            3.34m
         9           0.8905           0.0325            3.37m
        10           0.8546           0.0398            3.34m
        20           0.5679           0.0210            3.07m
        30           0.4389           0.0103            2.79m
        40           0.3438           0.0109            2.54m
        50           0.2765           0.0102            2.29m
        60           0.2320           0.0030            2.03m
        70           0.2048           0.0022            1.78m
        80           0.1769           0.0022            1.55m
        90           0.1606           0.0089            1.30m
       100           0.1488           0.0006            1.06m
Gradient Boosting Best Parameters: {'criterion': 'squared_error', 'learning_rate': 0.1009124836035503, 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 12, 'min_samples_split': 13, 'n_estimators': 138, 'random_state': 42, 'subsample': 0.645614570099021, 'verbose': 0}
INFO:__main__:Gradient Boosting - Train MCC Score: 0.9773
INFO:__main__:Gradient Boosting - Validation MCC Score: 0.1176
INFO:__main__:Time elapsed: 38103.22 seconds
INFO:__main__:Evaluating SVC
INFO:__main__:Skipping feature selection for SVC due to lack of support
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]SVC Best Parameters: {'C': 2.179416628681888, 'break_ties': False, 'cache_size': 142.42722776276352, 'class_weight': None, 'coef0': 8.422847745949985, 'decision_function_shape': 'ovo', 'degree': 3, 'gamma': 'auto', 'kernel': 'poly', 'max_iter': 100, 'probability': True, 'random_state': 42, 'shrinking': False, 'tol': 0.032664076880583544, 'verbose': False}
INFO:__main__:SVC - Train MCC Score: 0.0847
INFO:__main__:SVC - Validation MCC Score: 0.0279
INFO:__main__:Time elapsed: 43045.61 seconds
INFO:__main__:Evaluating KNN
INFO:__main__:Skipping feature selection for KNN due to lack of support

In [50]:
logger.info(f"KNN ran too long, stopped using keyboard interupt. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:KNN ran too long, stopped using keyboard interupt. Time elapsed: 153263.61 seconds


In [55]:
# Evaluate models
best_models = evaluate_models(train_df_transformed, y_train, validation_df_transformed, y_val)

logger.info(f"Model evaluation completed. Time elapsed: {time.time() - start_time:.2f} seconds")

INFO:__main__:Evaluating XGBoost
INFO:__main__:Skipping feature selection for XGBoost due to lack of support
[I 2024-08-11 18:17:28,818] A new study created in memory with name: no-name-01982fb4-7c7f-4a6b-8c70-48973a65a723
[I 2024-08-11 18:17:43,345] Trial 0 finished with value: 0.16645274069863777 and parameters: {'n_estimators': 131, 'max_depth': 7, 'learning_rate': 0.09691714162643293, 'subsample': 0.72985598738952, 'colsample_bytree': 0.8890304511176041, 'gamma': 0.02541938685174208, 'min_child_weight': 10}. Best is trial 0 with value: 0.16645274069863777.
[I 2024-08-11 18:17:58,659] Trial 1 finished with value: 0.23756922578754852 and parameters: {'n_estimators': 216, 'max_depth': 4, 'learning_rate': 0.0174599666871897, 'subsample': 0.8490776503324144, 'colsample_bytree': 0.7285445624992525, 'gamma': 0.010971740396791342, 'min_child_weight': 7}. Best is trial 1 with value: 0.23756922578754852.
[I 2024-08-11 18:18:09,434] Trial 2 finished with value: 0.05050432023649736 and paramet

XGBoost Best Parameters: {'n_estimators': 185, 'max_depth': 3, 'learning_rate': 0.030766375375650176, 'subsample': 0.7235689967656632, 'colsample_bytree': 0.8206070379962451, 'gamma': 0.1879894980741618, 'min_child_weight': 9}


INFO:__main__:XGBoost - Train MCC Score: 0.7314
INFO:__main__:XGBoost - Validation MCC Score: 0.3042
INFO:__main__:Time elapsed: 155428.53 seconds
INFO:__main__:Evaluating LightGBM
INFO:__main__:Skipping feature selection for LightGBM due to lack of support
[I 2024-08-11 18:37:35,290] A new study created in memory with name: no-name-0d83e76b-3800-43e6-a314-624ef95af077


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 925
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 15
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce MX250, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (26.75 MB) transferred to GPU in 0.064349 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


[I 2024-08-11 18:38:30,991] Trial 0 finished with value: 0.2247735354910986 and parameters: {'n_estimators': 252, 'learning_rate': 0.06978376146924166, 'num_leaves': 39, 'max_depth': 8, 'subsample': 0.6051973226857328, 'colsample_bytree': 0.8166771784783128, 'reg_alpha': 0.1397442075352253, 'reg_lambda': 0.020553668278924287}. Best is trial 0 with value: 0.2247735354910986.


...

[I 2024-08-11 18:57:36,965] Trial 48 finished with value: 0.25691209624258904 and parameters: {'n_estimators': 136, 'learning_rate': 0.07216494472309398, 'num_leaves': 36, 'max_depth': 1, 'subsample': 0.9755808462907565, 'colsample_bytree': 0.986083245760812, 'reg_alpha': 0.008153124452308272, 'reg_lambda': 0.15412998402845715}. Best is trial 31 with value: 0.34687568117829215.


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 925
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 15
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce MX250, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (26.75 MB) transferred to GPU in 0.074332 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

[I 2024-08-11 18:57:48,383] Trial 49 finished with value: 0.3378340470581193 and parameters: {'n_estimators': 108, 'learning_rate': 0.03254572895559675, 'num_leaves': 50, 'max_depth': 2, 'subsample': 0.9263102115428175, 'colsample_bytree': 0.9012494841907506, 'reg_alpha': 0.08218120346102245, 'reg_lambda': 0.14461937945068049}. Best is trial 31 with value: 0.34687568117829215.


LightGBM Best Parameters: {'n_estimators': 113, 'learning_rate': 0.060145056421709, 'num_leaves': 54, 'max_depth': 2, 'subsample': 0.972433563280902, 'colsample_bytree': 0.9585252901749111, 'reg_alpha': 0.05754570256074987, 'reg_lambda': 0.1698212335759648}
[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 925
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 15
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce MX250, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (26.75 MB) transferred to GPU in 0.084574 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110
[LightGBM

INFO:__main__:LightGBM - Train MCC Score: 0.5759
INFO:__main__:LightGBM - Validation MCC Score: 0.3469
INFO:__main__:Time elapsed: 156658.07 seconds
INFO:__main__:Model evaluation completed. Time elapsed: 156658.07 seconds


INFO:__main__:Evaluating Logistic Regression
INFO:__main__:Logistic Regression - Best Params: {'C': 374.54111884736244, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
INFO:__main__:Logistic Regression - MCC Score: 0.0486
INFO:__main__:Time elapsed: 1072.11 seconds
INFO:__main__:Evaluating Ridge Classifier
INFO:__main__:Ridge Classifier - Best Params: {'alpha': 3.845401188473625, 'fit_intercept': True, 'max_iter': 300, 'solver': 'cholesky'}
INFO:__main__:Ridge Classifier - MCC Score: 0.0506
INFO:__main__:Time elapsed: 1342.97 seconds
INFO:__main__:Evaluating Decision Tree
INFO:__main__:Decision Tree - Best Params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 17, 'max_features': 'log2', 'max_leaf_nodes': 17, 'min_impurity_decrease': 0.05986584841970366, 'min_samples_leaf': 0.025601864044243652, 'min_samples_split': 0.025599452033620268, 'min_weight_fraction_leaf': 0.005808361216819946, 'splitter': 'random'}
INFO:__main__:Decision Tree - MCC Score: 0.0000
INFO:__main__:Time elapsed: 1521.41 seconds
INFO:__main__:Evaluating Bagging Classifier

INFO:__main__:Evaluating Logistic Regression
INFO:__main__:Logistic Regression - Best Params: {'C': 374.54111884736244, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
INFO:__main__:Logistic Regression - MCC Score: 0.2327
INFO:__main__:Time elapsed: 5055.08 seconds
INFO:__main__:Evaluating Ridge Classifier
INFO:__main__:Ridge Classifier - Best Params: {'alpha': 3.845401188473625, 'fit_intercept': True, 'max_iter': 300, 'solver': 'cholesky'}
INFO:__main__:Ridge Classifier - MCC Score: 0.2332
INFO:__main__:Time elapsed: 5342.57 seconds
INFO:__main__:Evaluating Decision Tree
INFO:__main__:Decision Tree - Best Params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 17, 'max_features': 'log2', 'max_leaf_nodes': 17, 'min_impurity_decrease': 0.05986584841970366, 'min_samples_leaf': 0.025601864044243652, 'min_samples_split': 0.025599452033620268, 'min_weight_fraction_leaf': 0.005808361216819946, 'splitter': 'random'}
INFO:__main__:Decision Tree - MCC Score: 0.0000
INFO:__main__:Time elapsed: 5536.35 seconds
INFO:__main__:Evaluating Bagging Classifier
INFO:__main__:Bagging Classifier - Best Params: {'base_estimator': DecisionTreeClassifier(), 'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.5917173949330818, 'max_samples': 0.8898455001363846, 'n_estimators': 30, 'n_jobs': None, 'oob_score': False, 'random_state': 42}
INFO:__main__:Bagging Classifier - MCC Score: 0.9804
INFO:__main__:Time elapsed: 13779.59 seconds
INFO:__main__:Evaluating Random Forest
INFO:__main__:Random Forest - Best Params: {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 171, 'n_jobs': None, 'oob_score': True, 'random_state': 42, 'verbose': 0, 'warm_start': True}
INFO:__main__:Random Forest - MCC Score: 0.9818
INFO:__main__:Time elapsed: 23825.43 seconds
INFO:__main__:Evaluating AdaBoost Classifier
INFO:__main__:AdaBoost Classifier - Best Params: {'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(max_depth=1), 'learning_rate': 0.9607143064099162, 'n_estimators': 156, 'random_state': 42}
INFO:__main__:AdaBoost Classifier - MCC Score: 0.5079
INFO:__main__:Time elapsed: 54563.87 seconds
INFO:__main__:Evaluating Gradient Boosting

In [ ]:
# Assuming you have y_test for the test set
y_test = pd.read_csv('test_labels.csv', index_col='id')  # Load test labels if available
X_test_transformed = preprocessor.transform(test_df)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=all_feature_names)
test_results = test_best_models(best_models, X_test_transformed, y_test)

logger.info(f"All evaluations completed.")